In [1]:
#Weather data Viewer

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import folium
from rich import print
from rich.console import Console
from rich.table import Table
from rich.text import Text
from scipy.stats import linregress

# Impor the OpenWeatherMap API key
from api_keys import weather_api_key

# Import citipy to determine the cities based on latitude and longitude
from citipy import citipy

#Export cities.csv to db
from pymongo import MongoClient


In [3]:
#extract the data

In [4]:
#Generates a list of unique city names
# Empty list for holding the latitude and longitude combinations
lat_lngs = []

# Empty list for holding the cities names
cities = []

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name

    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
print(f"Number of cities in the list: {len(cities)}")

Number of cities in the list: 577

In [5]:
#Identify the API_key
weather_api_key


'5038cf4afa60a38fb2b6b761f6038892'

In [6]:
#API call
url = f'https://api.openweathermap.org/data/2.5/weather?q={city}&appid={weather_api_key}'


In [7]:
# check status
print(requests.get(url))


<Response [200]>

In [8]:
# check response
print(requests.get(url).ok)


True

In [9]:
#ETL

In [10]:
#retrive weather data
# Set the API base URL
url = f'https://api.openweathermap.org/data/2.5/weather?q={city}&appid={weather_api_key}'

# Define an empty list to fetch the weather data for each city
city_data = []

# Print to logger
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters
record_count = 1
set_count = 1

# Loop through all the cities in our list to fetch weather data
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 0

    # Create endpoint URL with each city
    city_url = f'https://api.openweathermap.org/data/2.5/weather?q={city}&appid={weather_api_key}&units=metric'# YOUR CODE HERE

    # Log the url, record, and set numbers
    print("Processing Record %s of Set %s | %s" % (record_count, set_count, city))

    # Add 1 to the record count
    record_count += 1

    # Run an API request for each of the cities
    try:
        # Parse the JSON and retrieve data
        city_weather = requests.get(city_url).json() # YOUR CODE HERE

        # Parse out latitude, longitude, max temp, humidity, cloudiness, wind speed, country, and date
        city_lat = city_weather['coord']['lat']# YOUR CODE HERE
        city_lng = city_weather['coord']['lon']# YOUR CODE HERE
        city_max_temp = city_weather['main']['temp_max']# YOUR CODE HERE
        city_humidity = city_weather['main']['humidity']# YOUR CODE HERE
        city_clouds = city_weather['clouds']['all']# YOUR CODE HERE
        city_wind = city_weather['wind']['speed']# YOUR CODE HERE
        city_country = city_weather['sys']['country']# YOUR CODE HERE
        city_date = city_weather['dt']# YOUR CODE HER

        # Append the City information into city_data list
        city_data.append({"City": city,
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date})

    # If an error is experienced, skip the city
    except:
        print("City not found. Skipping...")
        pass

    # pause to avoid rate limiting
    time.sleep(1)

#Indicate that Data Loading is complete
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval

-----------------------------

Processing Record 1 of Set 1 | rabo de peixe

Processing Record 2 of Set 1 | waitangi

Processing Record 3 of Set 1 | edinburgh of the seven seas

...

Processing Record 25 of Set 12 | agadez

Processing Record 26 of Set 12 | holualoa

-----------------------------

Data Retrieval Complete

-----------------------------

In [11]:
# Transform the cities weather data into a Pandas DataFrame
city_data_df = pd.DataFrame(city_data)

# Show Record Count
#city_data_df.count()

In [12]:
city_data_df = pd.DataFrame(city_data)

# Create a Rich Console object
console = Console()

# Function to print the record count with Rich
def print_record_count_with_rich(city_data_df):
    # Get the count of non-null values for each column
    count_data = city_data_df.count()

    # Create a rich table to display the counts in a neat format
    table = Table(title="[bold cyan]Record Count per Column[/bold cyan]", show_lines=True)
    table.add_column("Column", style="bold green")
    table.add_column("Count", style="bold yellow")

    # Add rows to the table
    for column, count in count_data.items():
        table.add_row(column, str(count))

    # Print the table
    console.print(table)

# Call the function to print the record count with Rich
print_record_count_with_rich(city_data_df)

   Record Count per   
        Column        
┏━━━━━━━━━━━━┳━━━━━━━┓
┃ Column     ┃ Count ┃
┡━━━━━━━━━━━━╇━━━━━━━┩
│ City       │ 551   │
├────────────┼───────┤
│ Lat        │ 551   │
├────────────┼───────┤
│ Lng        │ 551   │
├────────────┼───────┤
│ Max Temp   │ 551   │
├────────────┼───────┤
│ Humidity   │ 551   │
├────────────┼───────┤
│ Cloudiness │ 551   │
├────────────┼───────┤
│ Wind Speed │ 551   │
├────────────┼───────┤
│ Country    │ 551   │
├────────────┼───────┤
│ Date       │ 551   │
└────────────┴───────┘

In [13]:
city_data_df = city_data_df.rename(columns={
    'Max Temp': 'Maximum_Temperature',
    'Wind Speed': 'Wind_Speed_kmh'
})


In [14]:
# Create a Rich Console object
console = Console()

# Function to print DataFrame info using Rich
def print_info_with_rich(city_data_df):
    # Get the non-null count, data types, and memory usage
    count_data = city_data_df.count()
    dtype_data = city_data_df.dtypes
    memory_data = city_data_df.memory_usage(deep=True)

    # Create a rich table to display the DataFrame info
    table = Table(title="[bold cyan]DataFrame Info[/bold cyan]", show_lines=True)
    table.add_column("Column", style="bold green")
    table.add_column("Non-Null Count", style="bold yellow")
    table.add_column("Dtype", style="bold magenta")
    table.add_column("Memory Usage", style="bold blue")

    # Add rows to the table
    for column in city_data_df.columns:
        non_null_count = count_data[column]
        dtype = dtype_data[column]
        memory_usage = memory_data[column]
        
        # Add row to the table
        table.add_row(column, str(non_null_count), str(dtype), str(memory_usage))

    # Print the table
    console.print(table)

# Sample DataFrame (replace this with your actual city_data_df)
city_data_df2 = pd.DataFrame(city_data)

# Call the function to print the DataFrame info with Rich
print_info_with_rich(city_data_df)

                         DataFrame Info                          
┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━┓
┃ Column              ┃ Non-Null Count ┃ Dtype   ┃ Memory Usage ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━┩
│ City                │ 551            │ object  │ 31941        │
├─────────────────────┼────────────────┼─────────┼──────────────┤
│ Lat                 │ 551            │ float64 │ 4408         │
├─────────────────────┼────────────────┼─────────┼──────────────┤
│ Lng                 │ 551            │ float64 │ 4408         │
├─────────────────────┼────────────────┼─────────┼──────────────┤
│ Maximum_Temperature │ 551            │ float64 │ 4408         │
├─────────────────────┼────────────────┼─────────┼──────────────┤
│ Humidity            │ 551            │ int64   │ 4408         │
├─────────────────────┼────────────────┼─────────┼──────────────┤
│ Cloudiness          │ 551            │ int64   │ 4408         │
├─────────────────────┼────────────────┼─────────┼──────────────┤
│ Wind_Speed_kmh      │ 551            │ float64 │ 4408         │
├─────────────────────┼────────────────┼─────────┼──────────────┤
│ Country             │ 551            │ object  │ 28101        │
├─────────────────────┼────────────────┼─────────┼──────────────┤
│ Date                │ 551            │ int64   │ 4408         │
└─────────────────────┴────────────────┴─────────┴──────────────┘

In [15]:
city_data_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 551 entries, 0 to 550
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   City                 551 non-null    object 
 1   Lat                  551 non-null    float64
 2   Lng                  551 non-null    float64
 3   Maximum_Temperature  551 non-null    float64
 4   Humidity             551 non-null    int64  
 5   Cloudiness           551 non-null    int64  
 6   Wind_Speed_kmh       551 non-null    float64
 7   Country              551 non-null    object 
 8   Date                 551 non-null    int64  
dtypes: float64(4), int64(3), object(2)
memory usage: 38.9+ KB


In [16]:
# Convert datetime to readable date format
city_data_df['Date']=(pd.to_datetime(city_data_df['Date'],unit='s')).dt.date
city_data_df

,City,Lat,Lng,Maximum_Temperature,Humidity,Cloudiness,Wind_Speed_kmh,Country,Date
0,rabo de peixe,37.8000,-25.5833,17.73,88,75,8.23,PT,2025-01-08
1,waitangi,-43.9535,-176.5597,15.46,73,28,5.81,NZ,2025-01-08
2,edinburgh of the seven seas,-37.0676,-12.3116,17.66,98,83,5.26,SH,2025-01-08
3,nova sintra,14.8667,-24.7167,23.58,71,0,8.26,CV,2025-01-08
4,linqiong,30.4159,103.4609,5.61,77,83,0.36,CN,2025-01-08
...,...,...,...,...,...,...,...,...,...
546,champerico,14.3000,-91.9167,29.18,64,0,3.88,GT,2025-01-08
547,languyan,5.2618,120.0807,25.41,89,100,1.29,PH,2025-01-08
548,gardnerville ranchos,38.8882,-119.7413,7.99,43,20,0.89,US,2025-01-08
549,agadez,19.7500,10.2500,16.14,25,0,5.16,NE,2025-01-08


In [17]:
# Create a Rich Console object for city weather data
console = Console()

# Function to print the first few rows of the DataFrame with Rich
def print_df_with_rich(df):
    # Create a Rich Table object
    table = Table(title="City Weather Data")

    # Add columns dynamically based on the DataFrame
    for column in df.columns:
        table.add_column(column, justify="center", style="bold")

    # Add rows dynamically based on the DataFrame data
    for _, row in df.head().iterrows():
        table.add_row(*[str(value) for value in row])

    # Print the table
    console.print(table)

# Call the function to print the first few rows of the DataFrame with Rich
print_df_with_rich(city_data_df)

                                                 City Weather Data                                                 
┏━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━┓
┃     City     ┃   Lat    ┃    Lng    ┃ Maximum_Tem… ┃ Humidity ┃ Cloudiness ┃ Wind_Speed… ┃ Country ┃    Date    ┃
┡━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━┩
│   rabo de    │   37.8   │ -25.5833  │    17.73     │    88    │     75     │    8.23     │   PT    │ 2025-01-08 │
│    peixe     │          │           │              │          │            │             │         │            │
│   waitangi   │ -43.9535 │ -176.5597 │    15.46     │    73    │     28     │    5.81     │   NZ    │ 2025-01-08 │
│ edinburgh of │ -37.0676 │ -12.3116  │    17.66     │    98    │     83     │    5.26     │   SH    │ 2025-01-08 │
│  the seven   │          │           │              │          │            │             │         │            │
│     seas     │          │           │              │          │            │             │         │            │
│ nova sintra  │ 14.8667  │ -24.7167  │    23.58     │    71    │     0      │    8.26     │   CV    │ 2025-01-08 │
│   linqiong   │ 30.4159  │ 103.4609  │     5.61     │    77    │     83     │    0.36     │   CN    │ 2025-01-08 │
└──────────────┴──────────┴───────────┴──────────────┴──────────┴────────────┴─────────────┴─────────┴────────────┘

In [18]:
# Create a Rich Console object
console = Console()

# Function to print the DataFrame with Rich
def print_df_with_rich(df):
    # Create a Rich Table object
    table = Table(title="City Weather Data")

    # Add columns dynamically based on the DataFrame
    for column in df.columns:
        table.add_column(column, justify="center", style="bold")

    # Add rows dynamically based on the DataFrame data
    for _, row in df.iterrows():
        table.add_row(*[str(value) for value in row])

    # Print the table using Rich
    console.print(table)

# Call the function to print the DataFrame with Rich
print_df_with_rich(city_data_df.head(10))

                                                 City Weather Data                                                 
┏━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━┓
┃     City     ┃   Lat    ┃    Lng    ┃ Maximum_Tem… ┃ Humidity ┃ Cloudiness ┃ Wind_Speed… ┃ Country ┃    Date    ┃
┡━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━┩
│   rabo de    │   37.8   │ -25.5833  │    17.73     │    88    │     75     │    8.23     │   PT    │ 2025-01-08 │
│    peixe     │          │           │              │          │            │             │         │            │
│   waitangi   │ -43.9535 │ -176.5597 │    15.46     │    73    │     28     │    5.81     │   NZ    │ 2025-01-08 │
│ edinburgh of │ -37.0676 │ -12.3116  │    17.66     │    98    │     83     │    5.26     │   SH    │ 2025-01-08 │
│  the seven   │          │           │              │          │            │             │         │            │
│     seas     │          │           │              │          │            │             │         │            │
│ nova sintra  │ 14.8667  │ -24.7167  │    23.58     │    71    │     0      │    8.26     │   CV    │ 2025-01-08 │
│   linqiong   │ 30.4159  │ 103.4609  │     5.61     │    77    │     83     │    0.36     │   CN    │ 2025-01-08 │
│ yellowknife  │  62.456  │ -114.3525 │    -15.71    │    92    │    100     │    8.75     │   CA    │ 2025-01-08 │
│  burlington  │ 39.8835  │ -74.6413  │    -0.58     │    44    │     75     │    6.69     │   US    │ 2025-01-08 │
│   ushuaia    │  -54.8   │   -68.3   │    14.81     │    55    │     75     │    3.09     │   AR    │ 2025-01-08 │
│  saskatoon   │ 52.1168  │ -106.6345 │    -0.01     │    80    │     40     │     7.2     │   CA    │ 2025-01-08 │
│  bredasdorp  │ -34.5322 │  20.0403  │    17.82     │    80    │     0      │    1.46     │   ZA    │ 2025-01-08 │
└──────────────┴──────────┴───────────┴──────────────┴──────────┴────────────┴─────────────┴─────────┴────────────┘

In [19]:
city_data_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 551 entries, 0 to 550
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   City                 551 non-null    object 
 1   Lat                  551 non-null    float64
 2   Lng                  551 non-null    float64
 3   Maximum_Temperature  551 non-null    float64
 4   Humidity             551 non-null    int64  
 5   Cloudiness           551 non-null    int64  
 6   Wind_Speed_kmh       551 non-null    float64
 7   Country              551 non-null    object 
 8   Date                 551 non-null    object 
dtypes: float64(4), int64(2), object(3)
memory usage: 38.9+ KB


In [20]:
city_data_df = city_data_df.rename(columns={
    'Max Temp': 'Maximum_Temperature',
    'Wind_Speed': 'Wind_Speed_kmh'
})


In [21]:
# Define the desired column order
desired_order = ['Date', 'Country', 'City', 'Cloudiness', 'Humidity', 
                 'Maximum_Temperature', 'Wind_Speed_kmh', 'Lat', 'Lng']

# Reorder the DataFrame to match the desired column order
city_data_df = city_data_df[desired_order]

# Create a rich Console object
console = Console()

# Create a rich Table object with a title and enhanced styles
table = Table(title="City Weather Data (Ordered)", show_header=True, header_style="bold cyan", show_lines=True)

# Add columns dynamically based on the reordered DataFrame
for column in city_data_df.columns:
    table.add_column(column, style="bold magenta", justify="center")

# Add rows from the reordered DataFrame (only the first 10 rows)
for idx, row in city_data_df.head(10).iterrows():
    # Apply alternating row styles
    if idx % 2 == 0:
        table.add_row(*[str(value) for value in row], style="dim")
    else:
        table.add_row(*[str(value) for value in row])

# Print the table with rich formatting
console.print(table)


                                            City Weather Data (Ordered)                                            
┏━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━┓
┃    Date    ┃ Country ┃     City     ┃ Cloudiness ┃ Humidity ┃ Maximum_Tem… ┃ Wind_Speed… ┃   Lat    ┃    Lng    ┃
┡━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━┩
│ 2025-01-08 │   PT    │   rabo de    │     75     │    88    │    17.73     │    8.23     │   37.8   │ -25.5833  │
│            │         │    peixe     │            │          │              │             │          │           │
├────────────┼─────────┼──────────────┼────────────┼──────────┼──────────────┼─────────────┼──────────┼───────────┤
│ 2025-01-08 │   NZ    │   waitangi   │     28     │    73    │    15.46     │    5.81     │ -43.9535 │ -176.5597 │
├────────────┼─────────┼──────────────┼────────────┼──────────┼──────────────┼─────────────┼──────────┼───────────┤
│ 2025-01-08 │   SH    │ edinburgh of │     83     │    98    │    17.66     │    5.26     │ -37.0676 │ -12.3116  │
│            │         │  the seven   │            │          │              │             │          │           │
│            │         │     seas     │            │          │              │             │          │           │
├────────────┼─────────┼──────────────┼────────────┼──────────┼──────────────┼─────────────┼──────────┼───────────┤
│ 2025-01-08 │   CV    │ nova sintra  │     0      │    71    │    23.58     │    8.26     │ 14.8667  │ -24.7167  │
├────────────┼─────────┼──────────────┼────────────┼──────────┼──────────────┼─────────────┼──────────┼───────────┤
│ 2025-01-08 │   CN    │   linqiong   │     83     │    77    │     5.61     │    0.36     │ 30.4159  │ 103.4609  │
├────────────┼─────────┼──────────────┼────────────┼──────────┼──────────────┼─────────────┼──────────┼───────────┤
│ 2025-01-08 │   CA    │ yellowknife  │    100     │    92    │    -15.71    │    8.75     │  62.456  │ -114.3525 │
├────────────┼─────────┼──────────────┼────────────┼──────────┼──────────────┼─────────────┼──────────┼───────────┤
│ 2025-01-08 │   US    │  burlington  │     75     │    44    │    -0.58     │    6.69     │ 39.8835  │ -74.6413  │
├────────────┼─────────┼──────────────┼────────────┼──────────┼──────────────┼─────────────┼──────────┼───────────┤
│ 2025-01-08 │   AR    │   ushuaia    │     75     │    55    │    14.81     │    3.09     │  -54.8   │   -68.3   │
├────────────┼─────────┼──────────────┼────────────┼──────────┼──────────────┼─────────────┼──────────┼───────────┤
│ 2025-01-08 │   CA    │  saskatoon   │     40     │    80    │    -0.01     │     7.2     │ 52.1168  │ -106.6345 │
├────────────┼─────────┼──────────────┼────────────┼──────────┼──────────────┼─────────────┼──────────┼───────────┤
│ 2025-01-08 │   ZA    │  bredasdorp  │     0      │    80    │    17.82     │    1.46     │ -34.5322 │  20.0403  │
└────────────┴─────────┴──────────────┴────────────┴──────────┴──────────────┴─────────────┴──────────┴───────────┘

In [22]:
# Create a Rich Console object
console = Console()

# Function to print the DataFrame with Rich
def print_df_with_rich(df):
    # Create a Rich Table object
    table = Table(title="City Weather Data")

    # Add columns dynamically based on the DataFrame
    for column in df.columns:
        table.add_column(column, justify="center", style="bold")

    # Add rows dynamically based on the DataFrame data
    for _, row in df.iterrows():
        table.add_row(*[str(value) for value in row])

    # Print the table using Rich
    console.print(table)

# Call the function to print the DataFrame with Rich
print_df_with_rich(city_data_df.head(10))

                                                 City Weather Data                                                 
┏━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━┓
┃    Date    ┃ Country ┃     City     ┃ Cloudiness ┃ Humidity ┃ Maximum_Tem… ┃ Wind_Speed… ┃   Lat    ┃    Lng    ┃
┡━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━┩
│ 2025-01-08 │   PT    │   rabo de    │     75     │    88    │    17.73     │    8.23     │   37.8   │ -25.5833  │
│            │         │    peixe     │            │          │              │             │          │           │
│ 2025-01-08 │   NZ    │   waitangi   │     28     │    73    │    15.46     │    5.81     │ -43.9535 │ -176.5597 │
│ 2025-01-08 │   SH    │ edinburgh of │     83     │    98    │    17.66     │    5.26     │ -37.0676 │ -12.3116  │
│            │         │  the seven   │            │          │              │             │          │           │
│            │         │     seas     │            │          │              │             │          │           │
│ 2025-01-08 │   CV    │ nova sintra  │     0      │    71    │    23.58     │    8.26     │ 14.8667  │ -24.7167  │
│ 2025-01-08 │   CN    │   linqiong   │     83     │    77    │     5.61     │    0.36     │ 30.4159  │ 103.4609  │
│ 2025-01-08 │   CA    │ yellowknife  │    100     │    92    │    -15.71    │    8.75     │  62.456  │ -114.3525 │
│ 2025-01-08 │   US    │  burlington  │     75     │    44    │    -0.58     │    6.69     │ 39.8835  │ -74.6413  │
│ 2025-01-08 │   AR    │   ushuaia    │     75     │    55    │    14.81     │    3.09     │  -54.8   │   -68.3   │
│ 2025-01-08 │   CA    │  saskatoon   │     40     │    80    │    -0.01     │     7.2     │ 52.1168  │ -106.6345 │
│ 2025-01-08 │   ZA    │  bredasdorp  │     0      │    80    │    17.82     │    1.46     │ -34.5322 │  20.0403  │
└────────────┴─────────┴──────────────┴────────────┴──────────┴──────────────┴─────────────┴──────────┴───────────┘

In [23]:
#city  and country data transform in capital letter 
city_data_df['City'] = city_data_df['City'].str.title()
city_data_df['Country'] = city_data_df['Country'].str.upper()
# Initialize the rich console
console = Console()

# Function to print the DataFrame with Rich
def print_df_with_rich(df):
    # Create a Rich Table object
    table = Table(title="City Weather Data")

    # Add columns dynamically based on the DataFrame
    for column in df.columns:
        table.add_column(column, justify="center", style="bold")

    # Add rows dynamically based on the DataFrame data
    for _, row in df.iterrows():
        table.add_row(*[str(value) for value in row])

    # Print the table using Rich
    console.print(table)

# Call the function to print the modified DataFrame with Rich
print_df_with_rich(city_data_df.head(10))

                                                 City Weather Data                                                 
┏━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━┓
┃    Date    ┃ Country ┃     City     ┃ Cloudiness ┃ Humidity ┃ Maximum_Tem… ┃ Wind_Speed… ┃   Lat    ┃    Lng    ┃
┡━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━┩
│ 2025-01-08 │   PT    │   Rabo De    │     75     │    88    │    17.73     │    8.23     │   37.8   │ -25.5833  │
│            │         │    Peixe     │            │          │              │             │          │           │
│ 2025-01-08 │   NZ    │   Waitangi   │     28     │    73    │    15.46     │    5.81     │ -43.9535 │ -176.5597 │
│ 2025-01-08 │   SH    │ Edinburgh Of │     83     │    98    │    17.66     │    5.26     │ -37.0676 │ -12.3116  │
│            │         │  The Seven   │            │          │              │             │          │           │
│            │         │     Seas     │            │          │              │             │          │           │
│ 2025-01-08 │   CV    │ Nova Sintra  │     0      │    71    │    23.58     │    8.26     │ 14.8667  │ -24.7167  │
│ 2025-01-08 │   CN    │   Linqiong   │     83     │    77    │     5.61     │    0.36     │ 30.4159  │ 103.4609  │
│ 2025-01-08 │   CA    │ Yellowknife  │    100     │    92    │    -15.71    │    8.75     │  62.456  │ -114.3525 │
│ 2025-01-08 │   US    │  Burlington  │     75     │    44    │    -0.58     │    6.69     │ 39.8835  │ -74.6413  │
│ 2025-01-08 │   AR    │   Ushuaia    │     75     │    55    │    14.81     │    3.09     │  -54.8   │   -68.3   │
│ 2025-01-08 │   CA    │  Saskatoon   │     40     │    80    │    -0.01     │     7.2     │ 52.1168  │ -106.6345 │
│ 2025-01-08 │   ZA    │  Bredasdorp  │     0      │    80    │    17.82     │    1.46     │ -34.5322 │  20.0403  │
└────────────┴─────────┴──────────────┴────────────┴──────────┴──────────────┴─────────────┴──────────┴───────────┘

In [24]:
#Transfor the city and country starting with capital letter
# Create a rich Console object
console = Console()

# Create a rich Table object with a title and enhanced styles
table = Table(title="City and Country Data with Capital Letters", show_header=True, header_style="bold cyan", show_lines=True)

# Add columns dynamically from the DataFrame
for column in city_data_df.columns:
    table.add_column(column, style="bold magenta", justify="center")

# Add rows from the DataFrame (only the first 10 rows)
for idx, row in city_data_df.head(10).iterrows():
    # Apply alternating row styles
    if idx % 2 == 0:
        table.add_row(*[str(value) for value in row], style="dim")
    else:
        table.add_row(*[str(value) for value in row])

# Print the table with rich formatting
console.print(table)

                                    City and Country Data with Capital Letters                                     
┏━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━┓
┃    Date    ┃ Country ┃     City     ┃ Cloudiness ┃ Humidity ┃ Maximum_Tem… ┃ Wind_Speed… ┃   Lat    ┃    Lng    ┃
┡━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━┩
│ 2025-01-08 │   PT    │   Rabo De    │     75     │    88    │    17.73     │    8.23     │   37.8   │ -25.5833  │
│            │         │    Peixe     │            │          │              │             │          │           │
├────────────┼─────────┼──────────────┼────────────┼──────────┼──────────────┼─────────────┼──────────┼───────────┤
│ 2025-01-08 │   NZ    │   Waitangi   │     28     │    73    │    15.46     │    5.81     │ -43.9535 │ -176.5597 │
├────────────┼─────────┼──────────────┼────────────┼──────────┼──────────────┼─────────────┼──────────┼───────────┤
│ 2025-01-08 │   SH    │ Edinburgh Of │     83     │    98    │    17.66     │    5.26     │ -37.0676 │ -12.3116  │
│            │         │  The Seven   │            │          │              │             │          │           │
│            │         │     Seas     │            │          │              │             │          │           │
├────────────┼─────────┼──────────────┼────────────┼──────────┼──────────────┼─────────────┼──────────┼───────────┤
│ 2025-01-08 │   CV    │ Nova Sintra  │     0      │    71    │    23.58     │    8.26     │ 14.8667  │ -24.7167  │
├────────────┼─────────┼──────────────┼────────────┼──────────┼──────────────┼─────────────┼──────────┼───────────┤
│ 2025-01-08 │   CN    │   Linqiong   │     83     │    77    │     5.61     │    0.36     │ 30.4159  │ 103.4609  │
├────────────┼─────────┼──────────────┼────────────┼──────────┼──────────────┼─────────────┼──────────┼───────────┤
│ 2025-01-08 │   CA    │ Yellowknife  │    100     │    92    │    -15.71    │    8.75     │  62.456  │ -114.3525 │
├────────────┼─────────┼──────────────┼────────────┼──────────┼──────────────┼─────────────┼──────────┼───────────┤
│ 2025-01-08 │   US    │  Burlington  │     75     │    44    │    -0.58     │    6.69     │ 39.8835  │ -74.6413  │
├────────────┼─────────┼──────────────┼────────────┼──────────┼──────────────┼─────────────┼──────────┼───────────┤
│ 2025-01-08 │   AR    │   Ushuaia    │     75     │    55    │    14.81     │    3.09     │  -54.8   │   -68.3   │
├────────────┼─────────┼──────────────┼────────────┼──────────┼──────────────┼─────────────┼──────────┼───────────┤
│ 2025-01-08 │   CA    │  Saskatoon   │     40     │    80    │    -0.01     │     7.2     │ 52.1168  │ -106.6345 │
├────────────┼─────────┼──────────────┼────────────┼──────────┼──────────────┼─────────────┼──────────┼───────────┤
│ 2025-01-08 │   ZA    │  Bredasdorp  │     0      │    80    │    17.82     │    1.46     │ -34.5322 │  20.0403  │
└────────────┴─────────┴──────────────┴────────────┴──────────┴──────────────┴─────────────┴──────────┴───────────┘

In [25]:
# Transform Maximum_Temperature to Fahrenheit
city_data_df['Maximum_Temperature_F'] = city_data_df['Maximum_Temperature'] * 9/5 + 32

# Define the desired column order, including the new Fahrenheit column
desired_order = ['Date', 'Country', 'City', 'Cloudiness', 'Humidity', 
                 'Maximum_Temperature', 'Maximum_Temperature_F', 'Wind_Speed_kmh', 'Lat', 'Lng']

# Reorder the DataFrame
city_data_df = city_data_df[desired_order]

# Create a rich Console object
console = Console()

# Create a rich Table object with a title and enhanced styles
table = Table(title="City Weather Data (With Fahrenheit)", show_header=True, header_style="bold cyan", show_lines=True)

# Add columns dynamically based on the reordered DataFrame
for column in city_data_df.columns:
    table.add_column(column, style="bold magenta", justify="center")

# Add rows from the reordered DataFrame (only the first 10 rows)
for idx, row in city_data_df.head(10).iterrows():
    # Apply alternating row styles
    if idx % 2 == 0:
        table.add_row(*[str(value) for value in row], style="dim")
    else:
        table.add_row(*[str(value) for value in row])

# Print the table with rich formatting
console.print(table)

                                        City Weather Data (With Fahrenheit)                                        
┏━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┓
┃   Date    ┃ Country ┃   City    ┃ Cloudine… ┃ Humidity ┃ Maximum_… ┃ Maximum… ┃ Wind_Spe… ┃   Lat    ┃   Lng    ┃
┡━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━┩
│ 2025-01-… │   PT    │  Rabo De  │    75     │    88    │   17.73   │  63.914  │   8.23    │   37.8   │ -25.5833 │
│           │         │   Peixe   │           │          │           │          │           │          │          │
├───────────┼─────────┼───────────┼───────────┼──────────┼───────────┼──────────┼───────────┼──────────┼──────────┤
│ 2025-01-… │   NZ    │ Waitangi  │    28     │    73    │   15.46   │  59.828  │   5.81    │ -43.9535 │ -176.55… │
├───────────┼─────────┼───────────┼───────────┼──────────┼───────────┼──────────┼───────────┼──────────┼──────────┤
│ 2025-01-… │   SH    │ Edinburgh │    83     │    98    │   17.66   │  63.788  │   5.26    │ -37.0676 │ -12.3116 │
│           │         │  Of The   │           │          │           │          │           │          │          │
│           │         │   Seven   │           │          │           │          │           │          │          │
│           │         │   Seas    │           │          │           │          │           │          │          │
├───────────┼─────────┼───────────┼───────────┼──────────┼───────────┼──────────┼───────────┼──────────┼──────────┤
│ 2025-01-… │   CV    │   Nova    │     0     │    71    │   23.58   │ 74.4439… │   8.26    │ 14.8667  │ -24.7167 │
│           │         │  Sintra   │           │          │           │          │           │          │          │
├───────────┼─────────┼───────────┼───────────┼──────────┼───────────┼──────────┼───────────┼──────────┼──────────┤
│ 2025-01-… │   CN    │ Linqiong  │    83     │    77    │   5.61    │  42.098  │   0.36    │ 30.4159  │ 103.4609 │
├───────────┼─────────┼───────────┼───────────┼──────────┼───────────┼──────────┼───────────┼──────────┼──────────┤
│ 2025-01-… │   CA    │ Yellowkn… │    100    │    92    │  -15.71   │ 3.72199… │   8.75    │  62.456  │ -114.35… │
├───────────┼─────────┼───────────┼───────────┼──────────┼───────────┼──────────┼───────────┼──────────┼──────────┤
│ 2025-01-… │   US    │ Burlingt… │    75     │    44    │   -0.58   │  30.956  │   6.69    │ 39.8835  │ -74.6413 │
├───────────┼─────────┼───────────┼───────────┼──────────┼───────────┼──────────┼───────────┼──────────┼──────────┤
│ 2025-01-… │   AR    │  Ushuaia  │    75     │    55    │   14.81   │  58.658  │   3.09    │  -54.8   │  -68.3   │
├───────────┼─────────┼───────────┼───────────┼──────────┼───────────┼──────────┼───────────┼──────────┼──────────┤
│ 2025-01-… │   CA    │ Saskatoon │    40     │    80    │   -0.01   │  31.982  │    7.2    │ 52.1168  │ -106.63… │
├───────────┼─────────┼───────────┼───────────┼──────────┼───────────┼──────────┼───────────┼──────────┼──────────┤
│ 2025-01-… │   ZA    │ Bredasdo… │     0     │    80    │   17.82   │  64.076  │   1.46    │ -34.5322 │ 20.0403  │
└───────────┴─────────┴───────────┴───────────┴──────────┴───────────┴──────────┴───────────┴──────────┴──────────┘

In [26]:
# Sort the DataFrame by the City column alphabetically
city_data_df = city_data_df.sort_values(by='City')

# Create a rich Console object
console = Console()

# Create a rich Table object with a title and enhanced styles
table = Table(title="City Weather Data (Alphabetically Sorted)", show_header=True, header_style="bold cyan", show_lines=True)

# Add columns dynamically based on the sorted DataFrame
for column in city_data_df.columns:
    table.add_column(column, style="bold magenta", justify="center")

# Add rows from the sorted DataFrame (only the first 10 rows for brevity)
for idx, row in city_data_df.head(10).iterrows():
    # Apply alternating row styles
    if idx % 2 == 0:
        table.add_row(*[str(value) for value in row], style="dim")
    else:
        table.add_row(*[str(value) for value in row])

# Print the table with rich formatting
console.print(table)

                                     City Weather Data (Alphabetically Sorted)                                     
┏━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━┓
┃   Date    ┃ Country ┃   City    ┃ Cloudine… ┃ Humidity ┃ Maximum_… ┃ Maximum_… ┃ Wind_Spe… ┃   Lat   ┃   Lng    ┃
┡━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━┩
│ 2025-01-… │   AF    │ 'Alaqahd… │    23     │    53    │   7.02    │ 44.63599… │   2.09    │ 30.4374 │  63.313  │
│           │         │   Dishu   │           │          │           │           │           │         │          │
├───────────┼─────────┼───────────┼───────────┼──────────┼───────────┼───────────┼───────────┼─────────┼──────────┤
│ 2025-01-… │   GL    │  Aasiaat  │    100    │    95    │   -6.33   │  20.606   │   4.48    │ 68.7098 │ -52.8699 │
├───────────┼─────────┼───────────┼───────────┼──────────┼───────────┼───────────┼───────────┼─────────┼──────────┤
│ 2025-01-… │   MX    │ Acapulco  │     0     │    61    │   26.52   │  79.736   │   4.35    │ 16.8634 │ -99.8901 │
│           │         │ De Juarez │           │          │           │           │           │         │          │
├───────────┼─────────┼───────────┼───────────┼──────────┼───────────┼───────────┼───────────┼─────────┼──────────┤
│ 2025-01-… │   BR    │  Acarau   │    100    │    76    │   27.46   │  81.428   │   5.57    │ -2.8856 │  -40.12  │
├───────────┼─────────┼───────────┼───────────┼──────────┼───────────┼───────────┼───────────┼─────────┼──────────┤
│ 2025-01-… │   PN    │ Adamstown │    100    │    80    │   24.62   │  76.316   │   10.26   │ -25.066 │ -130.10… │
├───────────┼─────────┼───────────┼───────────┼──────────┼───────────┼───────────┼───────────┼─────────┼──────────┤
│ 2025-01-… │   PF    │ Afaahiti  │    66     │    75    │   26.5    │   79.7    │   5.31    │ -17.75  │ -149.28… │
├───────────┼─────────┼───────────┼───────────┼──────────┼───────────┼───────────┼───────────┼─────────┼──────────┤
│ 2025-01-… │   NE    │  Agadez   │     0     │    25    │   16.14   │  61.052   │   5.16    │  19.75  │  10.25   │
├───────────┼─────────┼───────────┼───────────┼──────────┼───────────┼───────────┼───────────┼─────────┼──────────┤
│ 2025-01-… │   GU    │   Agat    │    20     │    88    │   25.98   │  78.764   │   2.06    │ 13.3886 │ 144.6585 │
│           │         │  Village  │           │          │           │           │           │         │          │
├───────────┼─────────┼───────────┼───────────┼──────────┼───────────┼───────────┼───────────┼─────────┼──────────┤
│ 2025-01-… │   RU    │ Aginskoye │    56     │    97    │  -25.19   │ -13.3419… │   2.69    │  51.1   │  114.53  │
├───────────┼─────────┼───────────┼───────────┼──────────┼───────────┼───────────┼───────────┼─────────┼──────────┤
│ 2025-01-… │   NO    │ Akrehamn  │    75     │    93    │   0.14    │  32.252   │   2.06    │ 59.2667 │  5.1833  │
└───────────┴─────────┴───────────┴───────────┴──────────┴───────────┴───────────┴───────────┴─────────┴──────────┘

In [27]:
# Sort the DataFrame by the Country column alphabetically
city_data_df = city_data_df.sort_values(by='Country')

# Create a rich Console object
console = Console()

# Create a rich Table object with a title and enhanced styles
table = Table(title="City Weather Data (Sorted by Country)", show_header=True, header_style="bold cyan", show_lines=True)

# Add columns dynamically based on the sorted DataFrame
for column in city_data_df.columns:
    table.add_column(column, style="bold magenta", justify="center")

# Add rows from the sorted DataFrame (only the first 10 rows for brevity)
for idx, row in city_data_df.head(10).iterrows():
    # Apply alternating row styles
    if idx % 2 == 0:
        table.add_row(*[str(value) for value in row], style="dim")
    else:
        table.add_row(*[str(value) for value in row])

# Print the table with rich formatting
console.print(table)

                                       City Weather Data (Sorted by Country)                                       
┏━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━┓
┃   Date    ┃ Country ┃   City    ┃ Cloudine… ┃ Humidity ┃ Maximum_… ┃ Maximum_… ┃ Wind_Spe… ┃   Lat    ┃   Lng   ┃
┡━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━┩
│ 2025-01-… │   AF    │ 'Alaqahd… │    23     │    53    │   7.02    │ 44.63599… │   2.09    │ 30.4374  │ 63.313  │
│           │         │   Dishu   │           │          │           │           │           │          │         │
├───────────┼─────────┼───────────┼───────────┼──────────┼───────────┼───────────┼───────────┼──────────┼─────────┤
│ 2025-01-… │   AF    │   Parun   │     0     │    39    │  -13.66   │ 7.411999… │   8.58    │ 35.4207  │ 70.9221 │
├───────────┼─────────┼───────────┼───────────┼──────────┼───────────┼───────────┼───────────┼──────────┼─────────┤
│ 2025-01-… │   AF    │  Sangin   │     4     │    43    │   7.46    │  45.428   │   1.84    │ 32.0728  │ 64.8359 │
├───────────┼─────────┼───────────┼───────────┼──────────┼───────────┼───────────┼───────────┼──────────┼─────────┤
│ 2025-01-… │   AO    │  Malanje  │    80     │    96    │   18.03   │ 64.45400… │   1.99    │ -9.5402  │ 16.341  │
├───────────┼─────────┼───────────┼───────────┼──────────┼───────────┼───────────┼───────────┼──────────┼─────────┤
│ 2025-01-… │   AO    │ Benguela  │    82     │    79    │   24.41   │  75.938   │   2.29    │ -12.5763 │ 13.4055 │
├───────────┼─────────┼───────────┼───────────┼──────────┼───────────┼───────────┼───────────┼──────────┼─────────┤
│ 2025-01-… │   AO    │   Sumbe   │    62     │    84    │   24.73   │  76.514   │   3.38    │ -11.2061 │ 13.8437 │
├───────────┼─────────┼───────────┼───────────┼──────────┼───────────┼───────────┼───────────┼──────────┼─────────┤
│ 2025-01-… │   AO    │ Menongue  │    100    │    96    │   17.38   │  63.284   │   0.88    │ -14.6585 │ 17.691  │
├───────────┼─────────┼───────────┼───────────┼──────────┼───────────┼───────────┼───────────┼──────────┼─────────┤
│ 2025-01-… │   AO    │   Soio    │    40     │    85    │   26.38   │  79.484   │   4.46    │ -6.1349  │ 12.3689 │
├───────────┼─────────┼───────────┼───────────┼──────────┼───────────┼───────────┼───────────┼──────────┼─────────┤
│ 2025-01-… │   AO    │  Quibala  │    23     │    97    │   16.49   │  61.682   │   1.19    │ -10.7337 │  14.98  │
├───────────┼─────────┼───────────┼───────────┼──────────┼───────────┼───────────┼───────────┼──────────┼─────────┤
│ 2025-01-… │   AO    │  Namibe   │    100    │    79    │   21.54   │ 70.77199… │   1.47    │ -15.1961 │ 12.1522 │
└───────────┴─────────┴───────────┴───────────┴──────────┴───────────┴───────────┴───────────┴──────────┴─────────┘

In [75]:
# Export the City_Data into a csv
city_data_df.to_csv("Data_output/cities.csv", index_label="City_ID")


In [77]:
# Filter for northern cities
northern_cities_df = city_data_df[city_data_df['Lat'] > 0]

# Display the northern cities DataFrame
#northern_cities_df

In [79]:
# Define the desired column order, including the new Fahrenheit column
desired_order = ['Date', 'Country', 'City', 'Cloudiness', 'Humidity', 
                 'Maximum_Temperature', 'Maximum_Temperature_F', 'Wind_Speed_kmh', 'Lat', 'Lng']

# Reorder the DataFrame
northern_cities_df = northern_cities_df[desired_order]

In [81]:
# Export the northern_cities into a csv
northern_cities_df.to_csv("Data_output/northern_cities.csv", index_label="City_ID")
# Print a message confirming the file has been updated
print("northern_cities.csv has been successfully updated!")

northern_cities.csv has been successfully updated!

In [83]:
# Filter for southern cities
southern_cities_df = city_data_df[city_data_df['Lat'] < 0]

# Display the southern cities DataFrame
#northern_cities_df

In [85]:
# Export the northern_cities into a csv
southern_cities_df.to_csv("Data_output/southern_cities.csv", index_label="City_ID")
# Print a message confirming the file has been updated
print("southern_cities.csv has been successfully updated!")

southern_cities.csv has been successfully updated!